In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

In [2]:
#read in the csv data files containing historic return data for the universe of asset 
#classes I am planning to include in my global portfolio, along with data regarding the 
#market capitalisation based weights of each

asset_returns_orig = pd.read_csv('return_export.csv', index_col='Date', parse_dates=True)
asset_weights = pd.read_csv('asset_weights.csv', index_col='asset_class')
cols = ['SP500','US1.3.Year','US7.10.Year',
            'US20.Year','Aggregated.Bonds','IG.Corporate']

asset_returns = asset_returns_orig[cols]
asset_weights = asset_weights.loc[cols]

In [3]:
#mean return and variance of the global market portfolio

cov = asset_returns.cov()
print(asset_weights)
global_return = asset_returns.mean().multiply(asset_weights['weight'].values).sum()
market_var = np.matmul(asset_weights.values.reshape(len(asset_weights)).T,
                                       np.matmul(cov.values, asset_weights.values.reshape(len(asset_weights))))
print(f'The global market mean return is {global_return:.4f} and the variance is {market_var:.6}')
risk_aversion = global_return / market_var
print(f'The risk aversion parameter is {risk_aversion:.2f}')

                  weight
asset_class             
SP500              0.162
US1.3.Year         0.163
US7.10.Year        0.022
US20.Year          0.022
Aggregated.Bonds   0.030
IG.Corporate       0.299
The global market mean return is 0.0001 and the variance is 7.62403e-06
The risk aversion parameter is 8.39


In [4]:
#function which will help us reverse engineer the weights of a portfolio to obtain the Implied Equilibrium Return Vector.

def implied_rets(risk_aversion, sigma, w):
    
    implied_rets = risk_aversion * sigma.dot(w).squeeze()
    
    return implied_rets
implied_equilibrium_returns = implied_rets(risk_aversion, cov, asset_weights)
implied_equilibrium_returns

SP500               0.000208
US1.3.Year          0.000002
US7.10.Year         0.000014
US20.Year           0.000024
Aggregated.Bonds    0.000032
IG.Corporate        0.000094
Name: weight, dtype: float64

In [5]:
#this will differ depending on market cap weights

P = np.array(
    [
        [1, 0, 0, 0, 0, 0],
        [0, 1, -1, 0, 0, 0],
        [0, 0, 0, -1, 0.5, 0.5]
    ]
)

In [6]:
#variance of each individual portfolio view

view1_var = np.matmul(P[0].reshape(len(P[0])),np.matmul(cov.values, P[0].reshape(len(P[0])).T))
view2_var = np.matmul(P[1].reshape(len(P[1])),np.matmul(cov.values, P[1].reshape(len(P[1])).T))
view3_var = np.matmul(P[2].reshape(len(P[2])),np.matmul(cov.values, P[2].reshape(len(P[2])).T))
print(f'The Variance of View 1 Portfolio is {view1_var}, and the standard deviation is {np.sqrt(view1_var):.3f}\n',\
      f'The Variance of View 2 Portfolio is {view2_var}, and the standard deviation is {np.sqrt(view2_var):.3f}\n',\
      f'The Variance of View 3 Portfolio is {view3_var}, and the standard deviation is {np.sqrt(view3_var):.3f}')

The Variance of View 1 Portfolio is 0.00014972375581838123, and the standard deviation is 0.012
 The Variance of View 2 Portfolio is 1.2289246069080731e-05, and the standard deviation is 0.004
 The Variance of View 3 Portfolio is 5.4198410716643216e-05, and the standard deviation is 0.007


In [7]:
#covariance matrix of the error term

def error_cov_matrix(sigma, tau, P):
    matrix = np.diag(np.diag(P.dot(tau * cov).dot(P.T)))
    return matrix

tau = 0.1
omega = error_cov_matrix(cov, tau, P)


In [79]:
Q = [1, 0.1, 0.1]

In [80]:
#view based returns vector
sigma_scaled = cov * tau
BL_return_vector = implied_equilibrium_returns + sigma_scaled.dot(P.T).dot(inv(P.dot(sigma_scaled).dot(P.T) + omega).dot(Q - P.dot(implied_equilibrium_returns)))   

In [81]:
#compare the new return vector with the original Implied Return Vector below

returns_table = pd.concat([implied_equilibrium_returns, BL_return_vector], axis=1) * 100
returns_table.columns = ['Implied Returns', 'BL Return Vector']
returns_table['Difference'] = returns_table['BL Return Vector'] - returns_table['Implied Returns']

returns_table.style.format('{:,.4f}%')

,Implied Returns,BL Return Vector,Difference
SP500,0.0208%,49.7092%,49.6884%
US1.3.Year,0.0002%,-1.4168%,-1.4170%
US7.10.Year,0.0014%,-8.1350%,-8.1364%
US20.Year,0.0024%,-16.5593%,-16.5617%
Aggregated.Bonds,0.0032%,-2.3973%,-2.4005%
IG.Corporate,0.0094%,0.7687%,0.7593%


In [82]:
#calculate the new Black Litterman based weights vector
inverse_cov = pd.DataFrame(inv(cov.values), index=cov.columns, columns=cov.index)
BL_weights_vector = inverse_cov.dot(BL_return_vector)
BL_weights_vector = BL_weights_vector/sum(BL_weights_vector)

In [83]:
#We compare the new weights vector with the original Market Cap Weights below and the Mean-Variance optimised weights

MV_weights_vector = inverse_cov.dot(asset_returns.mean())
MV_weights_vector = MV_weights_vector/sum(MV_weights_vector)
weights_table = pd.concat([BL_weights_vector, asset_weights, MV_weights_vector], axis=1) * 100
weights_table.columns = ['BL Weights', 'Market Cap Weights', 'Mean-Var Weights']
weights_table['BL/Mkt Cap Diff'] = weights_table['BL Weights'] - weights_table['Market Cap Weights']

In [84]:
weights_table.style.format('{:,.2f}%')

,BL Weights,Market Cap Weights,Mean-Var Weights,BL/Mkt Cap Diff
asset_class,,,,
SP500,99.87%,16.20%,40.47%,83.67%
US1.3.Year,79.41%,16.30%,-3.13%,63.11%
US7.10.Year,-79.36%,2.20%,141.33%,-81.56%
US20.Year,31.51%,2.20%,-0.95%,29.31%
Aggregated.Bonds,-15.74%,3.00%,-45.59%,-18.74%
IG.Corporate,-15.68%,29.90%,-32.14%,-45.58%


In [85]:
#SP500 BL/Mkt Cap Diff for x=value
A1=weights_table['BL/Mkt Cap Diff'][0]
A2=weights_table['BL/Mkt Cap Diff'][1]
A3=weights_table['BL/Mkt Cap Diff'][2]
A4=weights_table['BL/Mkt Cap Diff'][3]
A5=weights_table['BL/Mkt Cap Diff'][4]
A6=weights_table['BL/Mkt Cap Diff'][5]

In [86]:
print(A1,',',A2,',',A3,',',A4,',',A5,',',A6)

83.6662745075851 , 63.10721727382004 , -81.56106201990625 , 29.308183574947886 , -18.743862785255764 , -45.57675055119104
